In [ ]:
from facenet_pytorch import MTCNN
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os
import pickle
import time

In [ ]:
# bbox has 4 elements, 1st and 2nd elements are top left x,y coordinates 
# 3rd and 4th elements are top bottom x,y coordinates
def show_img(img_path, bbox):
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    cv_img = np.array(img)
    x1 = int(bbox[0])
    y1 = int(bbox[1])
    x2 = int(bbox[2])
    y2 = int(bbox[3])
    cv2.rectangle(cv_img, (x1, y1), (x2, y2), (0,255,0), 3)
    plt.imshow(cv_img)
    plt.show()

In [ ]:
def rescale_box(box, scale):
    w = box[2] - box[0]
    h = box[3] - box[1]
    cx = box[0] + int(w/2)
    cy = box[1] + int(h/2)
    max_len = max(w, h)
    new_w = int(max_len * scale / 2)
    x1 = cx - new_w
    y1 = cy - new_w
    x2 = cx + new_w
    y2 = cy + new_w
    new_box = [x1,y1,x2,y2]
    return new_box

# crop image and resize to 224x224
def crop_img(img_path, coords, out_path):
    image_obj = Image.open(img_path)
    cropped_image = image_obj.crop(coords)
    img = cropped_image.resize((224,224))
    img.save(out_path)

# Test on a single image

In [ ]:
img_path = "/home/luqman/ME-GraphAU/data/DISFA/img/SN009/1760.png"
img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
frame = Image.fromarray(img)
mtcnn = MTCNN(keep_all=True, device='cuda:0')
boxes, probs = mtcnn.detect(frame)

In [ ]:
show_img(img_path, boxes[0])

In [ ]:
s_box = rescale_box(list(boxes[0]), 1.3)
show_img(img_path, s_box)

# Crop image

In [ ]:
start_time = time.time()
video_names = ['SN009']

box_path = '/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/DISFA/mtcnn'
input_dir = '/home/luqman/ME-GraphAU/data/DISFA/img'
output_path = '/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/DISFA/img'

for v in video_names:
    f_path = box_path + "/" + v + ".pickle"
    boxes = pickle.load(open(f_path, "rb"))
    img_dir = os.path.join(input_dir, v)
    
    out_dir = os.path.join(output_path, v)
    os.makedirs(out_dir, exist_ok=True)
    
    for i, box in enumerate(boxes):
        in_filename = str(i+1) + ".png"
        img_path = os.path.join(img_dir, in_filename)
        out_filename = str(i) + ".png"
        out_img_path = os.path.join(out_dir, out_filename)
        
        if (box is None):
            command = "cp " + img_path + " " + out_img_path  
            os.system(command)
        else:
            s_box = rescale_box(box[0], 1.3)
            crop_img(img_path, s_box, out_img_path)
            
print("--- %s seconds ---" % (time.time() - start_time))

# Count file in directory

In [47]:
DIR = '/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/DISFA/frames'
c = 0
for d in os.listdir(DIR):
    img_dir = os.path.join(DIR, d)
    l = len([name for name in os.listdir(img_dir)])
    print(f'{d} - {l}')
    c += 1
    print(c)

SN018 - 4845
1
SN008 - 4845
2
SN016 - 4845
3
SN001 - 4845
4
SN027 - 4845
5
SN031 - 4845
6
SN026 - 4845
7
SN032 - 4845
8
SN011 - 4845
9
SN003 - 4845
10
SN025 - 4845
11
SN004 - 4845
12
SN017 - 4845
13
SN028 - 4845
14
SN013 - 4845
15
SN012 - 4845
16
SN029 - 4845
17
SN021 - 4845
18
SN007 - 4845
19
SN024 - 4845
20
SN009 - 4845
21
SN010 - 4844
22
SN002 - 4845
23
SN006 - 4845
24
SN005 - 4845
25
SN023 - 4845
26
SN030 - 4845
27


In [ ]:
DIR = '/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/DISFA/mtcnn' 
for f in os.listdir(DIR):
    box_path = os.path.join(DIR, f)
    boxes = pickle.load(open(box_path, "rb"))
    print(f'{f} - {len(boxes)}')

In [ ]:
path = '/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/DISFA/mtcnn/SN009.pickle' 
boxes = pickle.load(open(path, "rb"))
counter = 0
for i, box in enumerate(boxes):
    if (box is None):
        counter += 1
        print(i)
print(counter)

# Run MTCNN

In [ ]:
start_time = time.time()
video_names = ['SN009']
input_dir = "/home/luqman/ME-GraphAU/data/DISFA/img"
batch_size = 32
mtcnn = MTCNN(keep_all=True, device='cuda:0')

for v in video_names:
    print(f'Processing {v}')
    frames = []
    boxes = []
    img_dir = os.path.join(input_dir, v)
    files_count = len([name for name in os.listdir(img_dir)])
    
    for i in range(files_count):
        filename = str(i+1) + '.png'
        img_path = os.path.join(img_dir, filename)
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(img)
        frames.append(frame)

        if (len(frames) == batch_size or (i+1 == files_count)):
            batch_boxes, _ = mtcnn.detect(frames)
            boxes.extend(batch_boxes)
            frames = []
                
    out_filename = v + ".pickle"
    out_path = os.path.join('/nas/project_data/B1_Behavior/hakim/ME-GraphAU/data/DISFA/mtcnn', out_filename)
    outfile = open(out_path, 'wb')
    pickle.dump(boxes, outfile)
    outfile.close()
    print(f'output is saved on {out_path}')
  
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
v_cap = cv2.VideoCapture('/nas/database/DISFA_Database/Videos_LeftCamera/LeftVideoSN009_comp.avi')
v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
batch_size = 32
frames = []
boxes = []

for i in range(v_len):    
    success, frame = v_cap.read()
    if not success:
        continue
        
    # Add to batch, resizing for speed
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = Image.fromarray(frame)
    frames.append(frame)
    
    # When batch is full, detect faces and reset batch list
    if len(frames) == batch_size or i == v_len:
        batch_boxes, _ = mtcnn.detect(frames)
        boxes.extend(batch_boxes)
        frames = []
        
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
boxes[1740]